# 1. Tải thư viện cần thiết

In [ ]:
# ==================== CELL 1: Optimized Installation ====================
print(" Installing dependencies...")

# Gỡ cài đặt cũ
!pip uninstall -y accelerate transformers peft -q

# Cập nhật pip
# !pip install -q --upgrade pip

# Cài đặt phiên bản tương thích
!pip install -q \
    "transformers==4.39.3" \
    "datasets==2.16.1" \
    "accelerate==0.27.2" \
    "evaluate==0.4.1" \
    "scikit-learn==1.3.2"

print(" Installation completed!")

In [ ]:

# ==================== CELL 2: Suppress Warnings ====================
import warnings
import os
import logging

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Tránh warning
logging.getLogger('tensorflow').setLevel(logging.ERROR)

print(" Warnings suppressed")

In [ ]:
# ==================== CELL 3: Import Libraries ====================
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import gc  # Garbage collector
from tqdm.auto import tqdm

# sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, confusion_matrix,
    classification_report, roc_curve, roc_auc_score
)

# Hugging Face
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    DataCollatorWithPadding
)
print(" Importing libraries...")

from datasets import Dataset, DatasetDict

# Kiểm tra thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n  Using device: {device}")

if torch.cuda.is_available():
    print(f" GPU: {torch.cuda.get_device_name(0)}")
    print(f" Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    torch.cuda.empty_cache()
else:
    print("  GPU not available, using CPU")

print("\nAll libraries imported successfully!")

# 2. Tạo lớp  XSSDataProcessor

In [ ]:
# ==================== CELL 4: Optimized Data Processor ====================
class XSSDataProcessor:
    """
    Xử lý tất cả việc tải, tăng cường và chuẩn bị dữ liệu.
    Version tối ưu với augmentation thông minh hơn.
    """
    def __init__(self):
        print("Khởi tạo XSSDataProcessor.")

   

    def prepare_dataset(self, train_df, test_df, val_size=0.1, random_state=42):
        """
        Chuẩn bị dataset từ các tập train và test đã chia sẵn, sau đó chia tập train thành Train/Validation.
    
        Args:
            train_df (pd.DataFrame): DataFrame chứa dữ liệu huấn luyện.
            test_df (pd.DataFrame): DataFrame chứa dữ liệu kiểm thử.
            val_size (float): Tỷ lệ phần trăm dữ liệu được tách ra làm tập Validation (từ train_df).
            random_state (int): Seed ngẫu nhiên để đảm bảo khả năng tái lập.
    
        Returns:
            tuple: (dataset, test_df) với dataset là DatasetDict (chứa 'train', 'validation', 'test') 
                   và test_df là tập kiểm thử ban đầu.
        """
        print("\n" + "="*60)
        print(" CHUẨN BỊ DỮ LIỆU TỪ TẬP ĐÃ CHIA")
        print("="*60)
        

    
        # 2. Chia Train thành Train và Validation
        train_df, val_df = train_test_split(
            train_df,
            test_size=0.1,
            random_state=random_state,
        )
    
        print(f"\n Tập Train (sau khi chia val): {len(train_df)} mẫu")
        print(f" Tập Validation: {len(val_df)} mẫu")
        print(f" Tập Test (đầu vào): {len(test_df)} mẫu")
    
        # 3. Chuyển đổi sang Hugging Face Dataset
        dataset = DatasetDict({
            'train': Dataset.from_pandas(train_df.reset_index(drop=True)),
            'validation': Dataset.from_pandas(val_df.reset_index(drop=True)),
            'test': Dataset.from_pandas(test_df.reset_index(drop=True))
        })
        
        # 4. In thống kê nhãn cho tập Test (đảm bảo tính công bằng)
        print("\n Thống kê nhãn tập Test:")
        try:
            test_stats = test_df['label'].value_counts(normalize=True).mul(100).round(2)
            print(test_stats.to_string())
        except KeyError:
            print("Không tìm thấy cột 'label' để thống kê.")
    
        return dataset, test_df

In [ ]:



# ==================== CELL 5: Optimized XSS Detector ====================
class XSSDetector:
    """
    Version tối ưu với better memory management và batch inference
    """
    def __init__(self, model_name, device, max_length=128):
        print("\n" + "="*60)
        print(f" KHỞI TẠO XSS DETECTOR VỚI MODEL: {model_name}")
        print("="*60)
        
        self.model_name = model_name
        self.device = device
        self.max_length = max_length
        self.trainer = None
        self.roc_auc = 0.0
        
        # Tải tokenizer và model
        print(f"Đang tải tokenizer cho: {self.model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        
        # Thêm pad token nếu không có (cho GPT2, CodeBERT)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        print(f"Đang tải mô hình cho: {self.model_name}...")
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=2,
            problem_type="single_label_classification",
            ignore_mismatched_sizes=True  # Important cho một số model
        )
        
        # Cấu hình pad token id cho model
        self.model.config.pad_token_id = self.tokenizer.pad_token_id
        self.model.to(self.device)
        
        print(f" Model đã tải: {self.model_name}")
        print(f" Tổng số tham số: {self.model.num_parameters():,}")
        
        # Clear cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    def _tokenize_function(self, examples):
        """Hàm token hóa với dynamic padding"""
        return self.tokenizer(
            examples['text'],
            truncation=True,
            max_length=self.max_length,
            # Bỏ padding ở đây, sẽ padding động trong DataCollator
        )

    def tokenize_dataset(self, dataset):
        """Token hóa dataset"""
        print("\n" + "="*60)
        print(" TOKEN HÓA DỮ LIỆU")
        print("="*60)
        
        tokenized_dataset = dataset.map(
            self._tokenize_function,
            batched=True,
            remove_columns=['text']  # Xóa column text để tiết kiệm memory
        )
        
        print(" Token hóa hoàn tất")
        return tokenized_dataset

    def _compute_metrics(self, eval_pred):
        """Tính toán metrics"""
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        
        accuracy = accuracy_score(labels, predictions)
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, predictions, average='binary', zero_division=0
        )
        
        cm = confusion_matrix(labels, predictions)
        if cm.size == 4:
            tn, fp, fn, tp = cm.ravel()
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        else:
            tp = fp = tn = fn = specificity = 0
        
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'specificity': specificity,
            'tp': int(tp),
            'fp': int(fp),
            'tn': int(tn),
            'fn': int(fn)
        }

    def train(self, tokenized_dataset, training_args):
        """Huấn luyện mô hình"""
        print("\n" + "="*60)
        print(" BẮT ĐẦU HUẤN LUYỆN")
        print("="*60 + "\n")
        
        # Sử dụng DataCollator cho dynamic padding
        data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)
        
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=tokenized_dataset['train'],
            eval_dataset=tokenized_dataset['validation'],
            tokenizer=self.tokenizer,
            data_collator=data_collator,  # Dynamic padding
            compute_metrics=self._compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        )
        
        # Clear cache trước khi train
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        train_result = self.trainer.train()
        
        print("\n✅ Huấn luyện hoàn tất!")
        print(f"Thời gian: {train_result.metrics['train_runtime']:.2f} giây")
        
        # Clear cache sau khi train
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

    def evaluate(self, eval_dataset=None):
        """Đánh giá mô hình"""
        if not self.trainer:
            print("Lỗi: Mô hình chưa được huấn luyện.")
            return None
            
        print("\n" + "="*60)
        print("ĐÁNH GIÁ MÔ HÌNH")
        print("="*60)
        
        eval_results = self.trainer.evaluate(eval_dataset=eval_dataset)

        print("\n📈 Kết quả:")
        for key, value in eval_results.items():
            if key.startswith('eval_'):
                metric_name = key.replace('eval_', '').upper()
                if isinstance(value, float):
                    print(f"  {metric_name}: {value:.4f}")
                else:
                    print(f"  {metric_name}: {value}")
        
        return eval_results

    def analyze_performance(self, tokenized_test_dataset):
        """Phân tích chi tiết với batch inference"""
        if not self.trainer:
            print("Lỗi: Mô hình chưa được huấn luyện.")
            return
            
        print("\n" + "="*60)
        print(" PHÂN TÍCH CHI TIẾT")
        print("="*60)

        # Batch prediction (nhanh hơn nhiều)
        predictions_output = self.trainer.predict(tokenized_test_dataset)
        
        # === LẤY THỜI GIAN SUY LUẬN TEST SET ===
        inference_time_test = predictions_output.metrics['test_runtime']
        samples_per_sec = predictions_output.metrics['test_samples_per_second']
        ms_per_sample = 1000 / samples_per_sec if samples_per_sec > 0 else 0
        
        print(f"\n⏱ Inference time (test set - {len(tokenized_test_dataset)} samples): {inference_time_test:.3f} giây")
        print(f"   Tốc độ: {samples_per_sec:.1f} mẫu/giây → {ms_per_sample:.2f} ms/mẫu")
        predictions = np.argmax(predictions_output.predictions, axis=1)
        labels = predictions_output.label_ids
        probabilities = torch.softmax(
            torch.tensor(predictions_output.predictions), dim=1
        ).numpy()

        # Classification Report
        print("\n Classification Report:")
        print(classification_report(
            labels, predictions,
            target_names=['Safe', 'XSS'],
            digits=4,
            zero_division=0
        ))

        # Confusion Matrix
        cm = confusion_matrix(labels, predictions)
        plt.figure(figsize=(8, 6))
        sns.heatmap(
            cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Safe', 'XSS'],
            yticklabels=['Safe', 'XSS']
        )
        plt.title(f'Confusion Matrix - {self.model_name}', fontweight='bold')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(f'cm_{self.model_name.replace("/", "_")}.png', dpi=300)
        plt.show()

        # ROC Curve
        if len(np.unique(labels)) > 1:
            fpr, tpr, _ = roc_curve(labels, probabilities[:, 1])
            self.roc_auc = roc_auc_score(labels, probabilities[:, 1])

            plt.figure(figsize=(8, 6))
            plt.plot(fpr, tpr, color='darkorange', lw=2,
                    label=f'ROC (AUC = {self.roc_auc:.4f})')
            plt.plot([0, 1], [0, 1], color='navy', lw=2,
                    linestyle='--', label='Random')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'ROC Curve - {self.model_name}', fontweight='bold')
            plt.legend(loc="lower right")
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.savefig(f'roc_{self.model_name.replace("/", "_")}.png', dpi=300)
            plt.show()
        else:
            print(" Chỉ có 1 class, bỏ qua ROC curve")
            self.roc_auc = 0.0

    def save_model(self, output_dir):
        """Lưu mô hình"""
        if not self.trainer:
            print("Lỗi: Mô hình chưa được huấn luyện.")
            return
            
        print("\n" + "="*60)
        print(" ĐANG LƯU MÔ HÌNH")
        print("="*60)
        
        model_path = f"{output_dir}/{self.model_name.replace('/', '_')}"
        os.makedirs(model_path, exist_ok=True)
        
        self.model.save_pretrained(model_path)
        self.tokenizer.save_pretrained(model_path)
        
        print(f" Mô hình đã lưu: {model_path}")
        return model_path

    def predict(self, text, show_details=True):
        """Dự đoán đơn lẻ"""
        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=self.max_length,
            padding=True
        )
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)
            pred = torch.argmax(probs, dim=1).item()
            conf = probs[0][pred].item()
        
        label = "🚨 XSS" if pred == 1 else "✅ SAFE"
        
        if show_details:
            print(f"\nInput: {text[:80]}...")
            print(f"Prediction: {label}")
            print(f"Confidence: {conf:.2%}")
            print("-" * 60)
            
        return pred, conf, label

    def batch_predict(self, texts, batch_size=32):
        """Batch prediction (nhanh hơn nhiều) + return thời gian"""
        self.model.eval()
        all_predictions = []
        all_confidences = []
    
        start_time = time.time()  
    
        for i in tqdm(range(0, len(texts), batch_size), desc="Predicting"):
            batch_texts = texts[i:i+batch_size]
            
            inputs = self.tokenizer(
                batch_texts,
                return_tensors='pt',
                truncation=True,
                max_length=self.max_length,
                padding=True
            )
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = self.model(**inputs)
                probs = torch.softmax(outputs.logits, dim=1)
                preds = torch.argmax(probs, dim=1)
                confs = probs[range(len(preds)), preds]
            
            all_predictions.extend(preds.cpu().numpy())
            all_confidences.extend(confs.cpu().numpy())
    
        end_time = time.time()
        total_time = end_time - start_time
        avg_time_per_sample = total_time / len(texts) if len(texts) > 0 else 0
    
        print(f"\n⏱ Tổng thời gian suy luận: {total_time:.3f} giây")
        print(f"   Trung bình / mẫu: {avg_time_per_sample*1000:.2f} ms")
    
        return all_predictions, all_confidences, total_time, avg_time_per_sample

    def analyze_errors(self, test_df):
        """Phân tích lỗi với batch inference"""
        print("\n" + "="*60)
        print(" PHÂN TÍCH LỖI")
        print("="*60)

        texts = test_df['text'].tolist()
        true_labels = test_df['label'].tolist()
        
        # Batch prediction
        predictions, confidences, *_ = self.batch_predict(texts)
        # Tìm errors
        errors = []
        for text, true, pred, conf in zip(texts, true_labels, predictions, confidences):
            if pred != true:
                errors.append({
                    'text': text,
                    'true': 'XSS' if true == 1 else 'Safe',
                    'pred': 'XSS' if pred == 1 else 'Safe',
                    'conf': conf
                })

        if errors:
            print(f"\n Tìm thấy {len(errors)} lỗi:\n")
            for i, err in enumerate(errors[:10], 1):
                print(f"{i}. Text: {err['text'][:60]}...")
                print(f"   True: {err['true']} | Pred: {err['pred']} | Conf: {err['conf']:.2%}\n")
        else:
            print("\n🎉 Không có lỗi!")


# 3. Hàm chạy thử nghiệm LLM

In [ ]:
# ==================== XSS COMPLEX PAYLOAD SUITE ====================

XSS_COMPLEX_SUITE = [
    # === 1. MULTI-LAYER ENCODING (Mã hóa đa lớp) ===
    {"payload": "&#x3c;&#x73;&#x63;&#x72;&#x69;&#x70;&#x74;&#x3e;alert(1)&#x3c;&#x2f;&#x73;&#x63;&#x72;&#x69;&#x70;&#x74;&#x3e;", "expected": 1, "type": "html_entity_original"},
    {"payload": "%253cscript%253ealert(1)%253c/script%253e", "expected": 1, "type": "double_url_original"},
    {"payload": "&lt;script&gt;alert(1)&lt;/script&gt;", "expected": 1, "type": "html_entity_display_original"},
    {"payload": "&#x3c;s&#x63;r&#x69;pt&#x3e;alert(1)&#x3c;/s&#x63;r&#x69;pt&#x3e;", "expected": 1, "type": "mixed_hex_entity"},
    {"payload": "%25253cscript%25253ealert(1)%25253c/script%25253e", "expected": 1, "type": "triple_url"},

    # === 2. JS OBFUSCATION (Che giấu mã JavaScript) ===
    {"payload": "eval(String.fromCharCode(97,108,101,114,116,40,49,41))", "expected": 1, "type": "fromcharcode_original"},
    {"payload": "eval(atob('YWxlcnQoMSk='))", "expected": 1, "type": "base64_original"},
    {"payload": "setTimeout('alert(1)', 0)", "expected": 1, "type": "settimeout_original"},
    {"payload": "window['al'+'ert'](1)", "expected": 1, "type": "window_concat"},
    {"payload": "new Function('alert(1)')()", "expected": 1, "type": "new_function"},
    {"payload": "alert(`${String.fromCharCode(88,83,83)}`)", "expected": 1, "type": "template_literal_fromcharcode"},

    # === 3. NULL BYTE & TRUNCATION (Byte Null và Cắt xén) ===
    {"payload": "<scr%00ipt>alert(1)</scr%00ipt>", "expected": 1, "type": "null_byte_original"},
    {"payload": "<script/src=data:,alert(1)>", "expected": 1, "type": "data_uri_original"},

    # === 4. EVENT HANDLER & TAG OBFUSCATION (Che giấu sự kiện và Thẻ) ===
    {"payload": "<img src=x onerror=eval('al'+'ert(1)')>", "expected": 1, "type": "string_concat_original"},
    {"payload": "<svg/onload=alert`1`>", "expected": 1, "type": "backtick_original"},
    {"payload": "<body onpageshow=alert(1)>", "expected": 1, "type": "rare_event_original"},
    {"payload": "<input autofocus onfocus=alert(1)>", "expected": 1, "type": "autofocus_onfocus"},
    {"payload": "<details open onpointerover=alert(1)>", "expected": 1, "type": "onpointerover"},

    # === 5. PROTOCOL & CSS (Giao thức và CSS) ===
    {"payload": "<a href=javascript:alert(1)>click", "expected": 1, "type": "javascript_uri_original"},
    {"payload": "<style>@import'javascript:alert(1)'</style>", "expected": 1, "type": "css_import_original"},
    {"payload": "<div style=background:url(javascript:alert(1))>", "expected": 1, "type": "css_url_original"},

    # === 6. DOM-BASED XSS (XSS dựa trên DOM) ===
    {"payload": "location.href = 'javascript:alert(1)'", "expected": 1, "type": "dom_location_original"},
    {"payload": "document.write('<script>alert(1)</script>')", "expected": 1, "type": "document_write_original"},
    {"payload": "innerHTML = '<img src=x onerror=alert(1)>'", "expected": 1, "type": "innerhtml_original"},

    # === 7. WAF BYPASS & MISC (Vượt tường lửa WAF và Khác) ===
    {"payload": "<script>/*!--*/alert(1)//</script>", "expected": 1, "type": "comment_obfuscation_original"},
    {"payload": "<scr<script>ipt>alert(1)</scr</script>ipt>", "expected": 1, "type": "tag_break_original"},
    {"payload": "<img src onerror=alert(1)>", "expected": 1, "type": "missing_quote_original"},
    {"payload": "<script>alert(1)</script>", "expected": 1, "type": "unicode_original"}, # Thường là payload cơ bản
    {"payload": "<img src=1 onerror=alert(String.fromCharCode(88,83,83))>", "expected": 1, "type": "fromcharcode_xss_original"},
    {"payload": "<img\tsrc=x onerror=alert(1)>", "expected": 1, "type": "tab_char"},
    {"payload": "<a href\n='javascript:alert(1)'>Click</a>", "expected": 1, "type": "newline_char"},
    {"payload": "<iframe src=javascript:alert(1)", "expected": 1, "type": "missing_closing_tag"},

    # === 8. FALSE POSITIVE TRAPS (SAFE - Expected: 0) (Bẫy Dương tính giả) ===
    {"payload": "<div>alert me</div>", "expected": 0, "type": "safe_text_original"},
    {"payload": "javascript is cool", "expected": 0, "type": "safe_word_original"},
    {"payload": "<img src='logo.png'>", "expected": 0, "type": "safe_img_original"},
    {"payload": "onload=loadImage()", "expected": 0, "type": "safe_event_original"},
    {"payload": "String.fromCharCode(65)", "expected": 0, "type": "safe_js_original"},
    {"payload": "User's feedback: I love this site! <Thank You>", "expected": 0, "type": "safe_mixed_text"},
    {"payload": "The product cost is 123.45$ (USD)", "expected": 0, "type": "safe_currency"},
    {"payload": "<p>This is a paragraph.</p>", "expected": 0, "type": "safe_html_tag"},
    {"payload": "<a href=\"/products/item?id=123\">View Item</a>", "expected": 0, "type": "safe_link_absolute"},
    {"payload": "", "expected": 0, "type": "safe_html_comment"},
    {"payload": "A script tag: &lt;script&gt; not run", "expected": 0, "type": "safe_escaped_script"},
    {"payload": "The function is called: 'alert(0)'", "expected": 0, "type": "safe_js_in_quote"},
    {"payload": "function test(a) { return a + 1; }", "expected": 0, "type": "safe_js_function"},
    {"payload": "document.getElementById('data').value", "expected": 0, "type": "safe_dom_access"},
]

In [ ]:
# ==================== MAIN EXPERIMENT RUNNER ====================
"""
Kịch bản thực nghiệm XSS Detection:
1. SCENARIO 1: Train trên RAW data (3 LLMs + Logistic Regression)
2. SCENARIO 2: Train trên AUGMENTED data (3 LLMs only)
3. Đánh giá trên test set và complex payloads cho tất cả models
"""

import pandas as pd
import numpy as np
import torch
import time
import json
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')



# ==================== EXPERIMENT CONFIGURATIONS ====================
MODELS_LLM = [
    "distilbert-base-uncased",
    "roberta-base",
    "microsoft/codebert-base"
]

DATA_CONFIG = {
    "raw_train": "/kaggle/input/xss-dataset/train.csv",
    "augmented_train": "/kaggle/input/xss-dataset/train_agumented.csv",
    "test": "/kaggle/input/xss-dataset/test.csv"
}

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LENGTH = 128
RESULTS_DIR = './experiment_results'

# ==================== EXPERIMENT TRACKER ====================
class ExperimentTracker:
    def __init__(self):
        self.results = {}

    def log(self, scenario, model_name, data_type, 
             test_metrics, payload_metrics, 
             train_time=None, 
             inference_test_s=None, inference_test_ms_per_sample=None,
             inference_payload_s=None, inference_payload_ms_per_sample=None):
        
        key = f"{model_name.split('/')[-1]}_{data_type}"  # đẹp hơn: codebert-base_augmented
        if scenario not in self.results:
            self.results[scenario] = {}
            
        self.results[scenario][key] = {
            "test_acc": round(test_metrics.get("accuracy",0), 4),
            "test_f1": round(test_metrics.get("f1",0), 4),
            "payload_acc": round(payload_metrics.get("accuracy",0), 4),
            "payload_f1": round(payload_metrics.get("f1",0), 4),
            "train_time_s": round(train_time, 1) if train_time else None,
            "inf_test_s": round(inference_test_s, 3) if inference_test_s else None,
            "inf_test_ms": round(inference_test_ms_per_sample, 2) if inference_test_ms_per_sample else None,
            "inf_payload_s": round(inference_payload_s, 3) if inference_payload_s else None,
            "inf_payload_ms": round(inference_payload_ms_per_sample, 2) if inference_payload_ms_per_sample else None,
        }
    def log_result(self, scenario, model_name, data_type, metrics):
        """Lưu kết quả thực nghiệm"""
        key = f"{model_name}_{data_type}"
        if scenario not in self.results:
            self.results[scenario] = {}
        self.results[scenario][key] = metrics
    def save(self, path="final_experiment_results.json"):
        with open(path, "w", encoding="utf-8") as f:
            json.dump(self.results, f, indent=2, ensure_ascii=False)
        print(f"\nĐã lưu kết quả vào {path}")
    def print_summary(self):
        print("\n" + "="*120)
        print("TÓM TẮT KẾT QUẢ THỰC NGHIỆM (bao gồm thời gian suy luận)")
        print("="*120)
        print(f"{'Model':<30} {'Data':<12} {'Test Acc':<10} {'Test F1':<10} {'Payload Acc':<12} {'Payload F1':<11} {'Train(s)':<10} {'Inf Test(s)':<12} {'ms/sample':<10} {'Inf Payload(s)':<10}")
        print("-" * 120)
        for scenario, models in self.results.items():
            for name, m in models.items():
                print(
                f"{name:<30} {name.split('_')[-1]:<12}"
                f"{m['test_acc']:<10.4f} {m['test_f1']:<10.4f} "
                f"{m['payload_acc']:<12.4f} {m['payload_f1']:<11.4f} "
                f"{m['train_time_s']:<10} {m['inf_test_s']:<12} "
                f"{m['inf_test_ms']:<10} {m['inf_payload_s']:<10}"
            )


In [ ]:
# ==================== LOGISTIC REGRESSION TRAINER ====================
def train_logistic_regression(X_train, y_train, X_test, y_test, payloads_df, tracker):
    """Train và đánh giá Logistic Regression"""
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.linear_model import LogisticRegression
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import GridSearchCV
    
    print("\n" + "="*80)
    print(" TRAINING LOGISTIC REGRESSION (RAW DATA)")
    print("="*80)
    
    # Pipeline
    pipe = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1,2))),
        ('clf', LogisticRegression(max_iter=1000, class_weight='balanced', solver='saga'))
    ])
    
    # Grid Search
    param_grid = [
        {'clf__penalty': ['l1'], 'clf__C': [0.01, 0.1, 1]},
        {'clf__penalty': ['l2'], 'clf__C': [0.01, 0.1, 1]},
    ]
    
    grid = GridSearchCV(pipe, param_grid, cv=3, scoring='f1', n_jobs=-1, verbose=1)
    
    # Training
    start = time.time()
    grid.fit(X_train, y_train)
    train_time = time.time() - start
    
    print(f"\n Training time: {train_time:.2f}s")
    print(f"Best params: {grid.best_params_}")
    
    # Evaluate on test set
    y_test_pred = grid.predict(X_test)
    test_metrics = {
        'accuracy': accuracy_score(y_test, y_test_pred),
        'report': classification_report(y_test, y_test_pred, output_dict=True),
        'f1': classification_report(y_test, y_test_pred, output_dict=True)['weighted avg']['f1-score']
    }
    
    print(f"\n Test Set Performance:")
    print(f"   Accuracy: {test_metrics['accuracy']:.4f}")
    print(f"   F1-Score: {test_metrics['f1']:.4f}")
    
    # Evaluate on complex payloads
    X_payloads = payloads_df['text']
    y_payloads = payloads_df['label']
    y_payload_pred = grid.predict(X_payloads)
    
    payload_metrics = {
        'accuracy': accuracy_score(y_payloads, y_payload_pred),
        'report': classification_report(y_payloads, y_payload_pred, output_dict=True),
        'f1': classification_report(y_payloads, y_payload_pred, output_dict=True)['weighted avg']['f1-score']
    }
    
    print(f"\n Complex Payloads Performance:")
    print(f"   Accuracy: {payload_metrics['accuracy']:.4f}")
    print(f"   F1-Score: {payload_metrics['f1']:.4f}")
    
    # Log results
    tracker.log_result('scenario1_raw', 'LogisticRegression', 'combined', {
        'test': test_metrics,
        'payload': payload_metrics,
        'train_time': train_time
    })
    
    return grid.best_estimator_


In [ ]:
# ==================== THÊM IMPORT NẾU CHƯA CÓ ====================
import time
import json
import os
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score
from datasets import Dataset, DatasetDict


# ==================== MAIN EXPERIMENT (ĐÃ CHỈNH SỬA HOÀN CHỈNH) ====================
def main():
    print("\n" + "="*90)
    print("XSS DETECTION EXPERIMENT")
    print("="*90)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    processor = XSSDataProcessor()
    tracker = ExperimentTracker()

    # Load dữ liệu
    train_raw_df = pd.read_csv("/kaggle/input/xss-dataset/train.csv")
    train_aug_df = pd.read_csv("/kaggle/input/xss-dataset/train_agumented.csv")  
    test_df = pd.read_csv("/kaggle/input/xss-dataset/test.csv")

    # Complex payload suite
    payloads_df = pd.DataFrame([
        { "text": item["payload"], "label": item["expected"] } for item in XSS_COMPLEX_SUITE
    ])

    print(f"Raw train: {len(train_raw_df):,} | Augmented train: {len(train_aug_df):,} | Test: {len(test_df):,}")

    # ===============================================
    # SCENARIO 1: RAW DATA (Logistic + 3 LLM)
    # ===============================================
    print("\n" + "="*90)
    print("SCENARIO 1: TRAINING ON RAW DATA")
    print("="*90)

    # Logistic Regression
    print("\nTraining Logistic Regression...")
    log_model = train_logistic_regression(
        train_raw_df["text"], train_raw_df["label"],
        test_df["text"], test_df["label"],
        payloads_df, tracker
    )

    # 3 LLM trên raw data
    for model_name in MODELS_LLM:
        print(f"\nTraining {model_name} trên RAW data...")
        start_time = time.time()

        dataset, _ = processor.prepare_dataset(train_raw_df.copy(), test_df.copy())

        detector = XSSDetector(model_name=model_name, device=device, max_length=128)

        tokenized = detector.tokenize_dataset(dataset)

        training_args = TrainingArguments(
            output_dir=f"./results/{model_name}_raw",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=3e-5,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=64,
            num_train_epochs=3,
            weight_decay=0.01,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            greater_is_better=True,
            save_total_limit=2,
            fp16=True,
            report_to="none",
            logging_steps=50,
            warmup_steps=100,
        )

        detector.train(tokenized, training_args)

        # Đánh giá trên test set
        test_results = detector.evaluate(eval_dataset=tokenized["test"])

        print("\nĐang đo thời gian suy luận + tính metrics trên test set...")
        predict_output = detector.trainer.predict(tokenized["test"])
        
        # Lấy metrics từ predict (có compute_metrics nên sẽ có test_accuracy, test_f1)
        test_accuracy = predict_output.metrics.get('test_accuracy', 0.0)
        test_f1       = predict_output.metrics.get('test_f1', 0.0)
        
        # Thời gian suy luận test set
        inference_test_s = predict_output.metrics['test_runtime']
        ms_per_sample_test = 1000 / predict_output.metrics['test_samples_per_second']
        
        print(f"✅ Test set: Acc = {test_accuracy:.4f}, F1 = {test_f1:.4f}")
        print(f"   Inference time: {inference_test_s:.3f}s → {ms_per_sample_test:.2f} ms/mẫu")

        # Vẫn chạy analyze_performance để vẽ biểu đồ (nó predict lại lần nữa nhưng chỉ mất ~2 giây)
        detector.analyze_performance(tokenized["test"])

        # ==================== COMPLEX PAYLOADS ====================
        print("\n" + "="*80)
        print(f"ĐÁNH GIÁ TRÊN COMPLEX PAYLOAD SUITE {len(XSS_COMPLEX_SUITE)} payload")
        print("="*80)

        # Chuyển payloads_df thành HuggingFace Dataset
        payload_dataset = Dataset.from_pandas(payloads_df.reset_index(drop=True))

        # Tokenize giống hệt train/test
        tokenized_payload = payload_dataset.map(
            detector._tokenize_function, batched=True, remove_columns=['text']
        )

        tokenized_payload.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

        # Dùng trainer.predict → có đầy đủ metrics + thời gian + confusion matrix + ROC tự động!
        payload_predict_output = detector.trainer.predict(tokenized_payload)

        payload_accuracy = payload_predict_output.metrics['test_accuracy']
        payload_f1       = payload_predict_output.metrics['test_f1']
        payload_time_s   = payload_predict_output.metrics['test_runtime']
        payload_ms       = 1000 / payload_predict_output.metrics['test_samples_per_second']

        print(f"Complex Payload Results → Acc: {payload_accuracy:.4f} | F1: {payload_f1:.4f}")
        print(f" Inference time {len(XSS_COMPLEX_SUITE)}: {payload_time_s:.3f}s → {payload_ms:.2f} ms/mẫu")

        # TỰ ĐỘNG vẽ Confusion Matrix + ROC Curve cho payload (lưu ảnh với tên riêng)
        print("\nĐang vẽ Confusion Matrix & ROC cho Complex Payload Suite...")
        old_model_name = detector.model_name  # backup tên model
        detector.model_name = f"{model_name.split('/')[-1]}_on_payload"  # đổi tạm tên để lưu ảnh khác
        detector.analyze_performance(tokenized_payload)  # sẽ lưu cm_..._on_payload.png và roc_..._on_payload.png
        detector.model_name = old_model_name  # trả lại tên gốc

        # Phân tích lỗi chi tiết (in ra payload nào bị sai)
        detector.analyze_errors(payloads_df)

        # ==================== LOG VÀO TRACKER (CẬP NHẬT CHÍNH XÁC) ====================
        tracker.log(
            scenario="scenario1_raw" if "raw" in training_args.output_dir else "scenario2_augmented",
            model_name=model_name,
            data_type="raw" if "raw" in training_args.output_dir else "augmented",
            test_metrics={"accuracy": test_accuracy, "f1": test_f1},
            payload_metrics={"accuracy": payload_accuracy, "f1": payload_f1},
            train_time=time.time() - start_time,
            inference_test_s=inference_test_s,
            inference_test_ms_per_sample=ms_per_sample_test,
            inference_payload_s=payload_time_s,
            inference_payload_ms_per_sample=payload_ms
        )

        # Lưu model tốt nhất
        detector.save_model("./saved_models")

        del detector, tokenized
        torch.cuda.empty_cache()
        gc.collect()

    # ===============================================
    # SCENARIO 2: AUGMENTED DATA (chỉ 3 LLM)
    # ===============================================
    print("\n" + "="*90)
    print("SCENARIO 2: TRAINING ON AUGMENTED DATA (LLM only)")
    print("="*90)

    for model_name in MODELS_LLM:
        print(f"\nTraining {model_name} trên AUGMENTED data...")
        start_time = time.time()

        dataset, _ = processor.prepare_dataset(train_aug_df.copy(), test_df.copy())

        detector = XSSDetector(model_name=model_name, device=device, max_length=128)

        tokenized = detector.tokenize_dataset(dataset)

        training_args = TrainingArguments(
            output_dir=f"./results/{model_name}_augmented",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=3e-5,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=64,
            num_train_epochs=3,
            weight_decay=0.01,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            greater_is_better=True,
            save_total_limit=2,
            fp16=True,
            report_to="none",
        )

        detector.train(tokenized, training_args)

        test_results = detector.evaluate(eval_dataset=tokenized["test"])
        print("\nĐang đo thời gian suy luận + tính metrics trên test set...")
        predict_output = detector.trainer.predict(tokenized["test"])
        
        # Lấy metrics từ predict (có compute_metrics nên sẽ có test_accuracy, test_f1)
        test_accuracy = predict_output.metrics.get('test_accuracy', 0.0)
        test_f1       = predict_output.metrics.get('test_f1', 0.0)
        
        # Thời gian suy luận test set
        inference_test_s = predict_output.metrics['test_runtime']
        ms_per_sample_test = 1000 / predict_output.metrics['test_samples_per_second']
        
        print(f"✅ Test set: Acc = {test_accuracy:.4f}, F1 = {test_f1:.4f}")
        print(f"   Inference time: {inference_test_s:.3f}s → {ms_per_sample_test:.2f} ms/mẫu")

        # Vẫn chạy analyze_performance để vẽ biểu đồ (nó predict lại lần nữa nhưng chỉ mất ~2 giây)
        detector.analyze_performance(tokenized["test"])
        
        # ==================== COMPLEX PAYLOADS ====================
        print("\n" + "="*80)
        print(f"ĐÁNH GIÁ TRÊN COMPLEX PAYLOAD SUITE{len(XSS_COMPLEX_SUITE)} payload")
        print("="*80)

        # Chuyển payloads_df thành HuggingFace Dataset
        payload_dataset = Dataset.from_pandas(payloads_df.reset_index(drop=True))

        # Tokenize giống hệt train/test
        tokenized_payload = payload_dataset.map(
            detector._tokenize_function, batched=True, remove_columns=['text']
        )

        tokenized_payload.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

        # Dùng trainer.predict → có đầy đủ metrics + thời gian + confusion matrix + ROC tự động!
        payload_predict_output = detector.trainer.predict(tokenized_payload)

        payload_accuracy = payload_predict_output.metrics['test_accuracy']
        payload_f1       = payload_predict_output.metrics['test_f1']
        payload_time_s   = payload_predict_output.metrics['test_runtime']
        payload_ms       = 1000 / payload_predict_output.metrics['test_samples_per_second']

        print(f"Complex Payload Results → Acc: {payload_accuracy:.4f} | F1: {payload_f1:.4f}")
        print(f" Inference time (22 payloads): {payload_time_s:.3f}s → {payload_ms:.2f} ms/mẫu")

        # TỰ ĐỘNG vẽ Confusion Matrix + ROC Curve cho payload (lưu ảnh với tên riêng)
        print("\nĐang vẽ Confusion Matrix & ROC cho Complex Payload Suite...")
        old_model_name = detector.model_name  # backup tên model
        detector.model_name = f"{model_name.split('/')[-1]}_on_payload"  # đổi tạm tên để lưu ảnh khác
        detector.analyze_performance(tokenized_payload)  # sẽ lưu cm_..._on_payload.png và roc_..._on_payload.png
        detector.model_name = old_model_name  # trả lại tên gốc

        # Phân tích lỗi chi tiết (in ra payload nào bị sai)
        detector.analyze_errors(payloads_df)

        # ==================== LOG VÀO TRACKER (CẬP NHẬT CHÍNH XÁC) ====================
        tracker.log(
            scenario="scenario1_raw" if "raw" in training_args.output_dir else "scenario2_augmented",
            model_name=model_name,
            data_type="raw" if "raw" in training_args.output_dir else "augmented",
            test_metrics={"accuracy": test_accuracy, "f1": test_f1},
            payload_metrics={"accuracy": payload_accuracy, "f1": payload_f1},
            train_time=time.time() - start_time,
            inference_test_s=inference_test_s,
            inference_test_ms_per_sample=ms_per_sample_test,
            inference_payload_s=payload_time_s,
            inference_payload_ms_per_sample=payload_ms
        )
        detector.save_model("./saved_models")
        del detector, tokenized
        torch.cuda.empty_cache()
        gc.collect()

    # ===============================================
    # KẾT QUẢ CUỐI
    # ===============================================
    # tracker.print_summary()
    # tracker.save("final_experiment_results_2025.json")

    print("\nHOÀN TẤT THỰC NGHIỆM! Tất cả model đã được lưu trong ./saved_models")

if __name__ == "__main__":
    main()